# Final cube analysis

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pickle

name_data_file = 'newExtrapolated_allCorrection.pickle'

with open(name_data_file, "rb") as input_file:
    data = pickle.load(input_file)

In [7]:
data.keys()

dict_keys(['thes', 'geoCUBE', 'dipCUBE', 'potCube', 'gams', 'kinCube', 'phis', 'smoCube'])